In [0]:
!pip install pyts

In [0]:
!pip install gcsfs

In [0]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.metrics
from tensorflow.keras.models import Model
from tensorflow.python.lib.io import file_io
import numpy as np
# import keras
import h5py
import pandas as pd
import re
import glob

from sklearn.metrics import classification_report
from scipy.io import loadmat
from pyts.image import GramianAngularField
import sys
from os import walk
from pyts.image import RecurrencePlot
import os
from google.cloud import storage
import gcsfs
from os import environ
from imblearn.over_sampling import (SMOTE, BorderlineSMOTE, SVMSMOTE, SMOTENC)
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import NearMiss 
from sklearn.model_selection import train_test_split


In [0]:
# model=tf.keras.models.load_model(r"G:\Seizure_Data\final\trainer\save_h5\model.h5")

In [0]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"/content/seizure-prediction123-593d1ad578b8.json"
print(os.environ.get('GOOGLE_APPLICATION_CREDENTIALS'))
client=storage.Client()
os.environ["GCP_BUCKET_NAME"] = "marwansrikarharris"
bucket_name = os.environ.get("GCP_BUCKET_NAME")
bucket = client.get_bucket(bucket_name)
files = bucket.list_blobs(prefix="Data/")
fileList = [file.name for file in files if '.' in file.name]

In [0]:
model_path=bucket.list_blobs(prefix="model/")

In [0]:
model_path=[file.name for file in model_path  if '.' in file.name]

In [0]:
model_path

In [0]:
fs=gcsfs.GCSFileSystem(project='seizure-prediction123')

In [0]:
# file_dir=bucket_name+r"/"+model_path[0]
# with fs.open(file_dir,'rb') as f:
#   mode=tf.keras.models.load_model(f)

In [0]:
# model_file=file.io.

In [0]:
model=tf.keras.models.load_model('/content/model_testfile1.h5')

In [0]:
def get_variables(x,path,searched_string):

#         print(test.group(0))
    searched_string=searched_string.group(0)
    searched_string=re.sub(r"[0]+(?=[1-9])","",searched_string)
    time_series_data=x[searched_string][0][0][0]
    data_length_sec=x[searched_string][0][0][1]
    sampling_frequency=data_length_sec=x[searched_string][0][0][2]
    channels=data_length_sec=x[searched_string][0][0][3]
#         sequence=data_length_sec=x[searched_string][0][0][4]
    assemble_matrix(time_series_data,path)
    

In [0]:
def convert_to_Gramiam(channel_data):
    transformer=RecurrencePlot()
    X_new = transformer.transform(channel_data)
    input_size=3996
    output_size=108
    bin_size = input_size // output_size
    X_new = X_new.reshape((1,output_size, bin_size,
                                   output_size, bin_size,)).max(4).max(2)
    # print("FINISHED RECURRENCE")
    X_new=X_new.astype(np.float16)
    return X_new

In [0]:
def assemble_matrix(time_series_data,path):
    time_series_section=time_series_data[:,3:239763]
    equal_partitions=np.hsplit(time_series_section,60)
    image_array=get_image_matrix(equal_partitions)
#     assemble_y(path)
    # assemble_dataframe(list_of_3D_arrays,path)
#     add_to_master(image_array)

In [0]:
def get_image_matrix(equal_partitions):
#     list_of_3D_array=[]
    for idx, segment in enumerate(equal_partitions):
        image_matrix=[]
        for index, single_channel in enumerate(segment,start=0):
            single_channel=np.reshape(single_channel,(1,len(single_channel)))
            image_matrix.append(convert_to_Gramiam(single_channel))
        master_array.append(create_3D_image_array(image_matrix))

In [0]:
def create_3D_image_array(image_matrix):
    image_1=image_matrix[0][0]
    image_2=image_matrix[1][0]
    image_3=image_matrix[2][0]
    image_4=image_matrix[3][0]
    image_5=image_matrix[4][0]
    image_6=image_matrix[5][0]
    image_7=image_matrix[6][0]
    image_8=image_matrix[7][0]
    image_9=image_matrix[8][0]
    image_10=image_matrix[9][0]
    image_11=image_matrix[10][0]
    image_12=image_matrix[11][0]
    image_13=image_matrix[12][0]
    image_14=image_matrix[13][0]
    image_15=image_matrix[14][0]
    # image_16=image_matrix[15][0]

    # image_array_3D=np.dstack([image_1,image_2,image_3,image_4,image_5,image_6,image_7,image_8,image_9,
    #                           image_10,image_11,image_12,image_13,image_14,image_15,image_16])
    image_array_3D=np.dstack([image_1,image_2,image_3,image_4,image_5,image_6,image_7,image_8,image_9,
                              image_10,image_11,image_12,image_13,image_14,image_15])

    return image_array_3D

In [0]:
# def add_to_master(list_of_3D_arrays):
#     sample_3D_array=list_of_3D_arrays
#     master_array.append(list_of_3D_arrays)

In [0]:
# def assemble_y(path):
#     match=re.search('preictal',path)
#     if match:
#         for i in range(60):
#             master_Y.append([0,1])
#     else:
#         for i in range(60):
#             master_Y.append([1,0])

In [0]:
def list_to_array():
    all_X=np.array(master_array)
    all_Y=np.array(master_Y)
    return all_X,all_Y

In [0]:
def assemble_y_test(label):
    if label==1:
        for i in range(60):
            master_Y.append([0,1])
    else:
        for i in range(60):
            master_Y.append([1,0])

In [0]:
# mode=tf.keras.models.load_model('/content/testfile1.h5')

In [0]:
def run(include_test=False):
#     data_files = glob.glob(r"/content/drive/My Drive/Seizure_Test_Data/*.mat")
    files=[]
    df=pd.read_csv(r"/content/SzPrediction_answer_key.csv")
    counter=0
    fs=gcsfs.GCSFileSystem(project='seizure-prediction123')
    for idx, file_name in enumerate(fileList):
        test=re.search(r'test',fileList[idx])
        
        if test:
            file_dir=bucket_name+r"/"+fileList[idx]
            with fs.open(file_dir,'rb') as f:
                master_array=[]
                master_Y=[]
                matching_string=r"(?<=[0-9]_)[\S]+(?=.mat)"
                test_subject_string=r"(?<=\/)Dog[0-9a-z_]+.mat"
                test_subject=re.search(test_subject_string,fileList[idx])
                searched_string=re.search(matching_string,fileList[idx])
                
                selected_rows=df.loc[df['clip']==test_subject.group(0)]
                               
                
                data_file=fileList[idx]
                
                
                if searched_string.group(0)=='test_segment_0031':
                  print("Filename: {} " .format(data_file))
                  print("ClassName: {} " .format(searched_string.group(0)))
                  assemble_y_test(selected_rows['preictal'].values[0]) 
                  x=loadmat(f)
                  get_variables(x,data_file,searched_string)
          
                  all_X,all_Y=list_to_array()
                  x_test=all_X
                  y_test=all_Y
                  Y_labels.append(y_test)
                  x_test=x_test.reshape(len(x_test),108,108,15,1)
                  prediction=mode.predict(x_test,batch_size=1)
                  predictions.append(prediction)
                # print("LOSS: {}" .format(loss))
                # print("ACCURACY: {}" .format(accuracy))
                # counter=counter+1
                  break
                counter=counter+1
                if counter==35:
                  break
                  
                

#     return all_X,all_Y


In [0]:
#  df=pd.read_csv(r"./SzPrediction_answer_key.csv")

In [0]:
# df.head()

In [0]:
predictions=[]
Y_labels=[]
master_array=[]
master_Y=[]
run(include_test=True)

In [0]:
test_predictions=predictions
array=np.array(test_predictions)
# type(test_predictions)
# array=array[9]
array[0].shape

In [0]:
# array=array.reshape(600,2)
# final_pred=[]
# final_pred=np.vstack((array[0],array[1]))
# for index, elements in enumerate(array):
#   if index+2<9:
#     final_pred=np.vstack((final_pred,array[index+2]))

In [0]:
final_pred=array[0]
final_pred.shape

(60, 2)

In [0]:
# len(Y_labels)
Y_array=np.array(Y_labels)
Y_array=Y_array[0]
Y_array.shape
# len(Y_labels)
# Y_labels[0].shape

In [0]:
final_pred[0:10]

In [0]:
test_predictions=final_pred
test_predictions=np.argmax(test_predictions,axis=1)
test_Y_pred=Y_array
test_Y_pred=np.argmax(test_Y_pred,axis=1)

In [0]:
test_predictions[0]

In [0]:
test_Y_pred[0]

In [0]:
classification_report(test_predictions,test_Y_pred)

In [0]:
final_pred[0:100]